In [2]:
import numpy as np
import pandas as pd
import torch
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


import sys, os


os.chdir("/Users/samuel.gruffaz/Documents/PEcollab")

from leaspy.api import Leaspy

from leaspy import Leaspy, Data, AlgorithmSettings, Plotter, Dataset, IndividualParameters
from leaspy.models.utils.attributes.attributes_factory import AttributesFactory
from leaspy.algo.algo_factory import AlgoFactory
from leaspy.utils.visualisation.visual_update_b import *

max benton 30
HVLT 60 MOCA 30 SCOPA-AUT 73 ADL 100


In [61]:

os.chdir("/Users/samuel.gruffaz/Documents/data")

dHVLT=pd.read_csv("HVLT.csv").set_index(['ID', 'TIME'])
dMOCA=pd.read_csv("MOCA.csv").set_index(['ID', 'TIME'])
dSCOPAAUT=pd.read_csv("SCOPA-AUT.csv").set_index(['ID', 'TIME'])
dBenton=pd.read_csv("Benton_JLO.csv").set_index(['ID', 'TIME'])
dADL=pd.read_csv("Schwab_and_England_ADL.csv").set_index(['ID', 'TIME'])

In [62]:
print(dHVLT.columns)
print(dMOCA.columns)
print(dSCOPAAUT.columns)
print(dBenton.columns)
print(dADL.columns)
dADL['SCHWAB_AND_ENGLAND']=1-dADL['SCHWAB_AND_ENGLAND']/100.
dBenton['BENTON_JLO']=1-dBenton['BENTON_JLO']/30.
dMOCA['MOCA_TOTAL']=1-dMOCA['MOCA_TOTAL']/30.
dHVLT['HVLT_TOTAL']=1-dHVLT['HVLT_TOTAL']/60.
dSCOPAAUT['SCOPA_TOTAL']=dSCOPAAUT['SCOPA_TOTAL']/73.0

Index(['HVLT_TOTAL', 'HVLT_1', 'HVLT_2', 'HVLT_3', 'HVLT_4', 'HVLT_5'], dtype='object')
Index(['MOCA_TOTAL'], dtype='object')
Index(['SCOPA_TOTAL', 'SCOPA_1', 'SCOPA_2', 'SCOPA_3', 'SCOPA_4', 'SCOPA_5',
       'SCOPA_6', 'SCOPA_7', 'SCOPA_8', 'SCOPA_9', 'SCOPA_10', 'SCOPA_11',
       'SCOPA_12', 'SCOPA_13', 'SCOPA_14', 'SCOPA_15', 'SCOPA_16', 'SCOPA_17',
       'SCOPA_18', 'SCOPA_19', 'SCOPA_20', 'SCOPA_21', 'SCOPA_22', 'SCOPA_23',
       'SCOPA_24', 'SCOPA_25', 'SCOPA_26'],
      dtype='object')
Index(['BENTON_JLO'], dtype='object')
Index(['SCHWAB_AND_ENGLAND'], dtype='object')


In [63]:
dHVLTtot=dHVLT.drop(['HVLT_1', 'HVLT_2', 'HVLT_3', 'HVLT_4',
       'HVLT_5'],axis=1)
dSCOPAAUTtot=dSCOPAAUT.drop(['SCOPA_1', 'SCOPA_2', 'SCOPA_3', 'SCOPA_4',
       'SCOPA_5', 'SCOPA_6', 'SCOPA_7', 'SCOPA_8', 'SCOPA_9', 'SCOPA_10',
       'SCOPA_11', 'SCOPA_12', 'SCOPA_13', 'SCOPA_14', 'SCOPA_15', 'SCOPA_16',
       'SCOPA_17', 'SCOPA_18', 'SCOPA_19', 'SCOPA_20', 'SCOPA_21', 'SCOPA_22',
       'SCOPA_23', 'SCOPA_24', 'SCOPA_25', 'SCOPA_26'],axis=1)


In [64]:
print(dSCOPAAUTtot.head(5))
print(dBenton.head(5))
print(dHVLT.head(10))
print(dMOCA.head(30))
print(dADL.head(30))

                SCOPA_TOTAL
ID   TIME                  
3000 70.123631     0.082192
     71.123631     0.123288
     72.123631     0.123288
     73.123631     0.109589
     74.123631     0.164384
                BENTON_JLO
ID   TIME                 
3000 70.123631    0.000000
     71.123631    0.000000
     72.123631    0.066667
     73.123631    0.133333
     74.123631    0.066667
                HVLT_TOTAL  HVLT_1  HVLT_2  HVLT_3  HVLT_4  HVLT_5
ID   TIME                                                         
3000 70.123631    0.133333     8.0    11.0    11.0    10.0    12.0
     71.123631    0.216667     7.0     9.0    10.0    10.0    11.0
     72.123631    0.166667     6.0    11.0    11.0    10.0    12.0
     73.123631    0.150000     7.0    10.0    11.0    11.0    12.0
     74.123631    0.133333     8.0    12.0    10.0    10.0    12.0
     75.123631    0.166667     9.0    10.0    10.0    10.0    11.0
     77.123631         NaN     5.0    10.0    11.0    10.0     NaN
     78.1236

In [65]:
dJoin=dHVLTtot.join(dSCOPAAUTtot,on=['ID','TIME'])
dJoin1=dJoin.join(dMOCA,on=['ID','TIME'])
dJoin2=dJoin1.join(dBenton,on=['ID','TIME'])
dJoin3=dJoin2.join(dADL,on=['ID','TIME'])

In [ ]:
df = df.set_index(['ID', 'TIME'])
indices = [idx for idx in df.index.unique('ID') if df.loc[idx].shape[0] >= 2]
df = df[df.index.get_level_values(0).isin(indices)]

In [38]:
print(dJoin3.head(4))

                HVLT_TOTAL  SCOPA_TOTAL  MOCA_TOTAL  BENTON_JLO  \
ID   TIME                                                         
3000 70.123631    0.866667     0.082192         NaN    1.000000   
     71.123631    0.783333     0.123288    0.966667    1.000000   
     72.123631    0.833333     0.123288    0.933333    0.933333   
     73.123631    0.850000     0.109589    1.000000    0.866667   

                SCHWAB_AND_ENGLAND  
ID   TIME                           
3000 70.123631                 NaN  
     71.123631                 NaN  
     72.123631                 NaN  
     73.123631                 NaN  


In [45]:
print(dJoin3.shape)

(6915, 5)


In [66]:
dJoinDropNan=dJoin3.dropna()

In [44]:
print(dJoinDropNan.shape)

(3301, 5)


In [67]:
dJoinDropNan.to_csv("Join5score.csv")